# Chapter8-3

## 最近傍探索ライブラリFaissを使った検索

SimCSEのモデルを用いて、類似分権策のウェブアプリを作成する

SimCSEモデルを用いて、テキストの集合の文埋め込みを獲得しFaissのインデックスを作成

ここでは日本語版wikipedia前記事の最初の段落のテキストの集合を用いる

### ライブラリのインストール

In [1]:
!pip install datasets faiss-cpu scipy transformers[ja,torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 65.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.3

### データセットの読み込み・前処理

ここでは日本語wikipediaの段落のテキストを文埋め込みの対象に用いる

In [2]:
from datasets import load_dataset

paragraph_dataset = load_dataset("llm-book/jawiki-paragraphs", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/9668476 [00:00<?, ? examples/s]

In [3]:
# データセットを確認
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 9668476
})


In [4]:
# データの内容を確認
from pprint import pprint

pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '5-89167474-1',
 'pageid': 5,
 'paragraph_index': 1,
 'revid': 89167474,
 'section': '語源',
 'text': '英語で教育を行う学校でアルファベットを復唱する場合、その文字自体が単語となる文字("A", "I", かつては "O" '
         'も)については、伝統的にラテン語の per se(それ自体)を用いて "A per se A" '
         'のように唱えられていた。また、アルファベットの最後に、27番目の文字のように "&" を加えることも広く行われていた。"&" '
         'はラテン語で et と読まれていたが、後に英語で and と読まれるようになった。結果として、アルファベットの復唱の最後は "X, Y, '
         'Z, and per se and" という形になった。この最後のフレーズが繰り返されるうちに "ampersand" '
         'と訛っていき、この言葉は1837年までには英語の一般的な語法となった。',
 'title': 'アンパサンド'}


In [5]:
# 段落データのうち、各記事の最初の段落のみを使うようにする
paragraph_dataset = paragraph_dataset.filter(
    lambda example: example["paragraph_index"] == 0
)

Filter:   0%|          | 0/9668476 [00:00<?, ? examples/s]

In [6]:
# フィルタリングの段落データの事例数を確認
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag'],
    num_rows: 1339236
})


In [7]:
# 内容を確認
pprint(paragraph_dataset[0])
pprint(paragraph_dataset[1])

{'html_tag': 'p',
 'id': '5-89167474-0',
 'pageid': 5,
 'paragraph_index': 0,
 'revid': 89167474,
 'section': '__LEAD__',
 'text': 'アンパサンド(&, 英語: '
         'ampersand)は、並立助詞「...と...」を意味する記号である。ラテン語で「...と...」を表す接続詞 "et" '
         'の合字を起源とする。現代のフォントでも、Trebuchet MS など一部のフォントでは、"et" '
         'の合字であることが容易にわかる字形を使用している。',
 'title': 'アンパサンド'}
{'html_tag': 'p',
 'id': '10-94194440-0',
 'pageid': 10,
 'paragraph_index': 0,
 'revid': 94194440,
 'section': '__LEAD__',
 'text': '言語(げんご)は、狭義には「声による記号の体系」をいう。',
 'title': '言語'}


### トークナイザ・モデルの準備

教師なしSimCSEのトークナイザと訓練済みのエンコーダを読み込む

In [8]:
from transformers import AutoModel, AutoTokenizer

model_name = "llm-book/bert-base-japanese-v3-unsup-simcse-jawiki"
tokenizer = AutoTokenizer.from_pretrained(model_name)
encoder = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

### 自分で学習した教師なしSimCSEを使うときは以下を実行する

In [9]:
from google.colab import drive

# Google Driveをマウント
drive.mount("drive")

Mounted at drive


In [10]:
!cp - drive/MyDrive/Learning-LLM/chapter8

cp: cannot stat '-': No such file or directory


In [12]:
from transformers import AutoModel, AutoTokenizer
import torch

# 保存された教師なしSimCSEのトークナイザ、エンコーダを読み込む
model_path = "/content/drive/MyDrive/Learning-LLM/encoder"
tokenizer = AutoTokenizer.from_pretrained(model_path)
encoder = AutoModel.from_pretrained(model_path)

# 読み込んだモデルをGPUのメモリに移動
device = "cuda" if torch.cuda.is_available() else "cpu"
encoder = encoder.to(device)


### モデルによる埋め込みの計算

SimCSEのエンコーダを使用し、テキストをベクトルに変換する

In [13]:
import numpy as np
import torch
import torch.nn.functional as F

def embed_texts(texts: list[str]) -> np.ndarray:
  """ SimCSEのモデルを用いてテキストの埋め込みを計算 """

  # トークナイザを適用
  tokenized_texts = tokenizer(
      texts,
      padding=True,
      truncation=True,
      max_length=128,
      return_tensors="pt",
  ).to(device)

  # 適用したテキストをベクトルに変換
  with torch.inference_mode():
    with torch.cuda.amp.autocast():
      encoded_texts = encoder(
          **tokenized_texts,
      ).last_hidden_state[:, 0]

  # ベクトルをNumpy配列に変換
  emb = encoded_texts.cpu().numpy().astype(np.float32)

  # ノルムが1になるように正規化
  emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)

  return emb

In [14]:
# 段落データのすべての事例に埋め込みを付与
paragraph_dataset = paragraph_dataset.map(
    lambda examples:{
        "embeddings": list(embed_texts(examples["text"]))
    },
    batched=True,
)

Parameter 'function'=<function <lambda> at 0x7986010f2950> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1339236 [00:00<?, ? examples/s]

In [15]:
# 保存したフィールドにベクトルが格納されていることを確認
print(paragraph_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})


In [16]:
# 事例を表示
pprint(paragraph_dataset[0])

{'embeddings': [-0.001463131164200604,
                -0.005663802847266197,
                0.02781076729297638,
                0.02374926581978798,
                -0.0025932935532182455,
                -0.03796450048685074,
                -0.01419072039425373,
                -0.00590280769392848,
                -0.02087251842021942,
                0.006345037836581469,
                0.011094599030911922,
                0.04039028659462929,
                -0.013257595710456371,
                -0.024490978568792343,
                -0.0225540641695261,
                -0.004168758634477854,
                0.012518559582531452,
                0.0034693656489253044,
                -0.009612899273633957,
                -0.005934076849371195,
                0.020100029185414314,
                -0.042298588901758194,
                0.0059973811730742455,
                -0.04281338304281235,
                0.0406823456287384,
                0.009434961713850498,
      

In [17]:
# 段落データを保存する
paragraph_dataset.save_to_disk("outputs_unsup_simcse/embedded_paragraphs")

Saving the dataset (0/10 shards):   0%|          | 0/1339236 [00:00<?, ? examples/s]

### Driveに保存する場合は以下を実行

In [18]:
!cp -r outputs_unsup_simcse/embedded_paragraphs drive/MyDrive/Learning_LLM/chapter8

### Faissによる最近傍探索

In [19]:
import faiss

# ベクトルの次元数を取り出す
emb_dim = encoder.config.hidden_size

# Faissインデックスを作成
index = faiss.IndexFlatIP(emb_dim)

# 段落データに保存されたフィールドのベクトルからFaissインデックスを構築
paragraph_dataset.add_faiss_index("embeddings", custom_index=index)

  0%|          | 0/1340 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'pageid', 'revid', 'paragraph_index', 'title', 'section', 'text', 'html_tag', 'embeddings'],
    num_rows: 1339236
})

In [22]:
# 任意のテキストに対して最近傍探索を実行

query_text = input("テキストを入力: ")

# 類似度上位10件の事例スコアを取得
scores, retrieved_examples = paragraph_dataset.get_nearest_examples(
    "embeddings", embed_texts([query_text])[0], k=10
)

# 取得した事例の内容をスコアとともに表示
titles = retrieved_examples["title"]
texts = retrieved_examples["text"]

for score, title, text in zip(scores, titles, texts):
  print(f"score: {score}  title: {title}  text: {text}")

テキストを入力: 日本語は、主に日本で話されている言語である
score: 0.9999963641166687  title: T-43  text: T-43は、以下のようなものの名称である。
score: 0.9999963045120239  title: マドラス (曖昧さ回避)  text: マドラス は、以下の語に用いられる言葉である。
score: 0.9999958872795105  title: 2000年代の建築  text: 2000年代の建築についての概要である。
score: 0.9999958276748657  title: ヒンドゥー教の宗派  text: ヒンドゥー教の宗派には、主に以下の4つがある。
score: 0.9999958276748657  title: ポルトガル語文学  text: ポルトガル語文学は、ポルトガル語による文学である。
score: 0.9999958276748657  title: 野田  text: 野田(のだ)は、日本で見られる地名、姓。
score: 0.9999958276748657  title: 洋画  text: 日本において、洋画(ようが)とは、
score: 0.999995768070221  title: 嵐 (曖昧さ回避)  text: 以下は、上記の自然現象にちなんだもの。
score: 0.9999957084655762  title: 2020年代の建築  text: 2020年代の建築についての概要である。
score: 0.9999957084655762  title: アイルランドの地理  text: アイルランドの地理では、アイルランド島の地理について記す。
